In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


>[Imports](#scrollTo=vIPHE_H8l5Gk)

>>[Library Imports](#scrollTo=-rUIjRYRl1n7)

>>[Data Imports](#scrollTo=c-z0XY5Bl9bY)

>[Car Dataset](#scrollTo=UyaGfEzuhpXs)

>>[Cars Data Extraction](#scrollTo=0yoTL_rBmtaP)

>[Data Exploration of Cars](#scrollTo=-EtZIRaSmyCt)

>[Preprocessing](#scrollTo=VnJQOWBAs_Tw)

>>[Data Cleaning](#scrollTo=5tgWkKh1uPy6)

>[Data Cleaning](#scrollTo=bFgPDoLhqJTs)

>[Data Transformation](#scrollTo=dXKtF3GB41x4)

>[Dataset Split](#scrollTo=Arv0qtXo59YG)

>[Classification](#scrollTo=cNTI3PugRMQT)

>>[Decision Tree](#scrollTo=CXwa1Uj08xVP)

>>>[Using Information Gain as attribute selection measure](#scrollTo=aVpK6hdHCuEc)

>>>>[Evaluation](#scrollTo=zy-y1ACfG96c)

>>>[Using Gini Index as Attribute Selection Measure](#scrollTo=TE0LA5jPC07I)

>>>>[Evaluation](#scrollTo=L6FjNz7GRUX6)

>>[Naive Bayes](#scrollTo=87s1P66hDMMn)

>>>[using k-fold cross validation](#scrollTo=NV2d62poFUAv)

>>[ANN](#scrollTo=a5kaVwigIQ61)

>>[Evaluation](#scrollTo=m9ZJ2M3jRnEa)



# Imports

## Library Imports

In [2]:
import os
from pathlib import Path
# Preprocessing
import numpy as np
import pandas as pd

# Data Viz
import plotly.express as px
import plotly.graph_objects as go

## Data Imports

In [3]:
DRIVE_FOLDER = Path().cwd() / "drive/MyDrive/"
DM_PROJECT_FOLDER = DRIVE_FOLDER / "DM Mini Project 2023/"

DATASET_FOLDER = DM_PROJECT_FOLDER / "DATASETS"
HEART_STAT_FILENAME = Path("heart_statlog_cleveland_hungary_final.csv")
HEART_DISEASE = Path("heart+disease/cleveland.data")
CAR_EVAL = Path("car+evaluation/")

DATASET_PATH = DATASET_FOLDER / HEART_STAT_FILENAME

In [4]:
origData = pd.read_csv(DATASET_PATH)

# Car Dataset

## Cars Data Extraction

Attribute Values:



   buying       v-high, high, med, low

   maint        v-high, high, med, low

   doors        2, 3, 4, 5-more

   persons      2, 4, more

   lug_boot     small, med, big

   safety       low, med, high

In [5]:
carFolder = DATASET_FOLDER / CAR_EVAL
carFolderFiles = [item for item in carFolder.iterdir()]

with open(carFolderFiles[1], 'r') as f:
  for line in f.readlines():
    print(line)

1. Title: Car Evaluation Database



2. Sources:

   (a) Creator: Marko Bohanec

   (b) Donors: Marko Bohanec   (marko.bohanec@ijs.si)

               Blaz Zupan      (blaz.zupan@ijs.si)

   (c) Date: June, 1997



3. Past Usage:



   The hierarchical decision model, from which this dataset is

   derived, was first presented in 



   M. Bohanec and V. Rajkovic: Knowledge acquisition and explanation for

   multi-attribute decision making. In 8th Intl Workshop on Expert

   Systems and their Applications, Avignon, France. pages 59-78, 1988.



   Within machine-learning, this dataset was used for the evaluation

   of HINT (Hierarchy INduction Tool), which was proved to be able to

   completely reconstruct the original hierarchical model. This,

   together with a comparison with C4.5, is presented in



   B. Zupan, M. Bohanec, I. Bratko, J. Demsar: Machine learning by

   function decomposition. ICML-97, Nashville, TN. 1997 (to appear)



4. Relevant Information Paragraph:



   C

In [6]:
carsDfOrig = pd.read_csv(carFolderFiles[-1], names = ["buying","maint","doors", "persons", "lug_boot", "safety", "CAR"])

In [7]:
carsDf = carsDfOrig.copy()
carsDf

,buying,maint,doors,persons,lug_boot,safety,CAR
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [8]:
def resetCarsDf():
  global carsDfOrig, carsDf
  carsDf = carsDfOrig.copy()
  return

# Data Exploration of Cars

In [9]:
carsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   CAR       1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [10]:
carsDf.head()

,buying,maint,doors,persons,lug_boot,safety,CAR
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


1. There are no missing values
2. All 6 are categorical variables
3. The target is CAR attribute

In [11]:
px.histogram(carsDf, x = "CAR", text_auto = True)

Since, we are doing a classificatin task.
We make use of 3 algorithms:
1. Naive Bayes,
2. ANN,
3. Decision Tree

# Preprocessing

Major Tasks in Data Preprocessing:
1. Data Cleaning
2. Data Integration
3. Data Reduction
4. Data Transformation and Data Discretization

## Data Cleaning
Tasks:
1. Handle Missing Values,
2. Smooth noisy data,
3. Identify or remove outliers,
4. Resolve inconsistencies
5. Detect and remove redundancies

In [12]:
carsDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   CAR       1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


No missing Values

# Data Cleaning

In [13]:
resetCarsDf()
carsDfBuyingAndMaintMap = {
    "vhigh": 4,
    "high": 3,
    "med": 2,
    "low": 1
}
carsDf.buying = carsDf.buying.apply(lambda x: carsDfBuyingAndMaintMap[x])
carsDf.maint = carsDf.maint.apply(lambda x: carsDfBuyingAndMaintMap[x])

In [14]:
carsDf

,buying,maint,doors,persons,lug_boot,safety,CAR
0,4,4,2,2,small,low,unacc
1,4,4,2,2,small,med,unacc
2,4,4,2,2,small,high,unacc
3,4,4,2,2,med,low,unacc
4,4,4,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,1,1,5more,more,med,med,good
1724,1,1,5more,more,med,high,vgood
1725,1,1,5more,more,big,low,unacc
1726,1,1,5more,more,big,med,good


In [15]:
carsDfLugBootMap = {
    "small": 1,
    "med": 2,
    "big": 3,
}
carsDfSafetyMap = {
    "low": 1,
    "med": 2,
    "high": 3
}

carsDf.lug_boot = carsDf.lug_boot.apply(lambda x: carsDfLugBootMap[x])
carsDf.safety = carsDf.safety.apply(lambda x: carsDfSafetyMap[x])


In [16]:
carsDf['persons'] = carsDfOrig['persons'].apply(lambda val: 6 if val == "more" else val)
# carsDf['persons'].value_counts()
carsDf['doors'] = carsDf['doors'].apply(lambda val: 5 if val == "5more" else val)


In [17]:
carsDf

,buying,maint,doors,persons,lug_boot,safety,CAR
0,4,4,2,2,1,1,unacc
1,4,4,2,2,1,2,unacc
2,4,4,2,2,1,3,unacc
3,4,4,2,2,2,1,unacc
4,4,4,2,2,2,2,unacc
...,...,...,...,...,...,...,...
1723,1,1,5,6,2,2,good
1724,1,1,5,6,2,3,vgood
1725,1,1,5,6,3,1,unacc
1726,1,1,5,6,3,2,good


In [18]:
carsDf.CAR.unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [19]:
order_CAR = ['unacc', 'acc', 'good', 'vgood']
CAR_data = pd.Categorical(carsDf['CAR'], categories = order_CAR, ordered = True)
carsDf[carsDf.columns[-1]] = CAR_data.codes
# carsDf_changedTarget = carsDf.CAR.apply(lambda val: )

# Data Transformation

In [20]:
# we do min-max normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columnsToNormalize = carsDf.columns[:-1]

carsDf[columnsToNormalize] = scaler.fit_transform(carsDf[columnsToNormalize])
carsDf

,buying,maint,doors,persons,lug_boot,safety,CAR
0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,1.0,1.0,0.0,0.0,0.0,0.5,0
2,1.0,1.0,0.0,0.0,0.0,1.0,0
3,1.0,1.0,0.0,0.0,0.5,0.0,0
4,1.0,1.0,0.0,0.0,0.5,0.5,0
...,...,...,...,...,...,...,...
1723,0.0,0.0,1.0,1.0,0.5,0.5,2
1724,0.0,0.0,1.0,1.0,0.5,1.0,3
1725,0.0,0.0,1.0,1.0,1.0,0.0,0
1726,0.0,0.0,1.0,1.0,1.0,0.5,2


In [21]:
carsDf

,buying,maint,doors,persons,lug_boot,safety,CAR
0,1.0,1.0,0.0,0.0,0.0,0.0,0
1,1.0,1.0,0.0,0.0,0.0,0.5,0
2,1.0,1.0,0.0,0.0,0.0,1.0,0
3,1.0,1.0,0.0,0.0,0.5,0.0,0
4,1.0,1.0,0.0,0.0,0.5,0.5,0
...,...,...,...,...,...,...,...
1723,0.0,0.0,1.0,1.0,0.5,0.5,2
1724,0.0,0.0,1.0,1.0,0.5,1.0,3
1725,0.0,0.0,1.0,1.0,1.0,0.0,0
1726,0.0,0.0,1.0,1.0,1.0,0.5,2


# Dataset Split

We use:
1. J48 for decision tree classifier
2. Multilayer perceptron
3. Naive Bayesian

for classification
***
For evaluation:

we do
1. train_test_ split
2. k-folds cross validation

In [22]:
# 60, 40 train_test_split

from sklearn.model_selection import train_test_split
X = carsDf.iloc[:, :-1]
y = carsDf.iloc[:, -1]
testSplitAt = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSplitAt, random_state = 42)


In [23]:
from sklearn.preprocessing import LabelEncoder
labeled_y_train = LabelEncoder().fit_transform(y_train)
labeled_y_train

array([0, 0, 0, ..., 1, 0, 1])

# Classification

## Apriori Algorithm

In [24]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=4d78217737a5c4490fa0587552934743baabfb8d5f310f02c5a6eedc7073da57
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [25]:
from apyori import apriori

In [26]:
transactions = carsDf.values.tolist()

results = list(
    apriori(transactions, min_support = 0.1, min_confidence = 0.5)
)
for result in results:
    print(result.items, 'Support:', result.support, 'Confidence:', result.ordered_statistics[0].confidence)

frozenset({0.0}) Support: 0.9195601851851852 Confidence: 0.9195601851851852
frozenset({0.5}) Support: 0.7037037037037037 Confidence: 0.7037037037037037
frozenset({1.0}) Support: 0.8576388888888888 Confidence: 0.8576388888888888
frozenset({0.0, 0.3333333333333333}) Support: 0.37037037037037035 Confidence: 0.8465608465608465
frozenset({0.0, 0.33333333333333337}) Support: 0.22511574074074073 Confidence: 0.9004629629629629
frozenset({0.0, 0.5}) Support: 0.6336805555555556 Confidence: 0.6336805555555556
frozenset({0.0, 0.6666666666666666}) Support: 0.2222222222222222 Confidence: 0.8888888888888888
frozenset({0.0, 0.6666666666666667}) Support: 0.3975694444444444 Confidence: 0.9087301587301587
frozenset({0.0, 0.9999999999999999}) Support: 0.2222222222222222 Confidence: 0.8888888888888888
frozenset({0.0, 1.0}) Support: 0.7771990740740741 Confidence: 0.7771990740740741
frozenset({0.3333333333333333, 0.5}) Support: 0.30787037037037035 Confidence: 0.7037037037037036
frozenset({0.3333333333333333,

## Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


### Using Information Gain as attribute selection measure

In [28]:
classifierModel = DecisionTreeClassifier(criterion = "entropy") # default is "gini"
classifierModel.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [29]:
y_pred = classifierModel.predict(X_test)

#### Evaluation

In [30]:
accuracy = accuracy_score(y_test, y_pred)

In [31]:
accuracy

0.9696531791907514

In [32]:
dTreeReportIG = pd.DataFrame(classification_report(y_test, y_pred, output_dict = True))
dTreeReportIG

,0,1,2,3,accuracy,macro avg,weighted avg
precision,0.997908,0.960526,0.764706,0.785714,0.969653,0.877214,0.971429
recall,0.993750,0.935897,0.896552,0.814815,0.969653,0.910253,0.969653
f1-score,0.995825,0.948052,0.825397,0.800000,0.969653,0.892318,0.970272
support,480.000000,156.000000,29.000000,27.000000,0.969653,692.000000,692.000000


In [33]:
dTreeReportIG.T

,precision,recall,f1-score,support
0,0.997908,0.993750,0.995825,480.000000
1,0.960526,0.935897,0.948052,156.000000
2,0.764706,0.896552,0.825397,29.000000
3,0.785714,0.814815,0.800000,27.000000
accuracy,0.969653,0.969653,0.969653,0.969653
macro avg,0.877214,0.910253,0.892318,692.000000
weighted avg,0.971429,0.969653,0.970272,692.000000


In [34]:
print(dTreeReportIG.T.style.to_latex())

\begin{tabular}{lrrrr}
 & precision & recall & f1-score & support \\
0 & 0.997908 & 0.993750 & 0.995825 & 480.000000 \\
1 & 0.960526 & 0.935897 & 0.948052 & 156.000000 \\
2 & 0.764706 & 0.896552 & 0.825397 & 29.000000 \\
3 & 0.785714 & 0.814815 & 0.800000 & 27.000000 \\
accuracy & 0.969653 & 0.969653 & 0.969653 & 0.969653 \\
macro avg & 0.877214 & 0.910253 & 0.892318 & 692.000000 \\
weighted avg & 0.971429 & 0.969653 & 0.970272 & 692.000000 \\
\end{tabular}



In [35]:
# using 10 fold
from sklearn.model_selection import cross_val_score
k = 10
cross_val_results = cross_val_score(classifierModel, X, y, cv = k)
for fold, accuracy in enumerate(cross_val_results, 1):
  print(f"Fold {fold}: Accuracy = {accuracy: .2f}")
print(f"Average Accuracy: {cross_val_results.mean():.2f}")

Fold 1: Accuracy =  0.75
Fold 2: Accuracy =  0.79
Fold 3: Accuracy =  0.83
Fold 4: Accuracy =  0.98
Fold 5: Accuracy =  0.90
Fold 6: Accuracy =  0.95
Fold 7: Accuracy =  0.92
Fold 8: Accuracy =  0.93
Fold 9: Accuracy =  0.91
Fold 10: Accuracy =  0.86
Average Accuracy: 0.88


### Using Gini Index as Attribute Selection Measure

In [36]:
classifierModelGini = DecisionTreeClassifier(criterion = "gini")
classifierModelGini.fit(X_train, y_train)

DecisionTreeClassifier()

In [37]:
y_pred = classifierModelGini.predict(X_test)

#### Evaluation

In [38]:
accuracy = accuracy_score(y_test, y_pred)

In [39]:
accuracy

0.9696531791907514

In [40]:
dTreeReportGini = pd.DataFrame(classification_report(y_test, y_pred, output_dict = True))
print(dTreeReportGini.T.style.to_latex())

\begin{tabular}{lrrrr}
 & precision & recall & f1-score & support \\
0 & 0.997912 & 0.995833 & 0.996872 & 480.000000 \\
1 & 0.979452 & 0.916667 & 0.947020 & 156.000000 \\
2 & 0.702703 & 0.896552 & 0.787879 & 29.000000 \\
3 & 0.800000 & 0.888889 & 0.842105 & 27.000000 \\
accuracy & 0.969653 & 0.969653 & 0.969653 & 0.969653 \\
macro avg & 0.870017 & 0.924485 & 0.893469 & 692.000000 \\
weighted avg & 0.973657 & 0.969653 & 0.970837 & 692.000000 \\
\end{tabular}



In [41]:
from sklearn.model_selection import cross_val_score
k = 10
cross_val_results = cross_val_score(classifierModelGini, X, y, cv = k)
for fold, accuracy in enumerate(cross_val_results, 1):
  print(f"Fold {fold}: Accuracy = {accuracy: .2f}")
print(f"Average Accuracy: {cross_val_results.mean():.2f}")

Fold 1: Accuracy =  0.96
Fold 2: Accuracy =  0.79
Fold 3: Accuracy =  0.83
Fold 4: Accuracy =  0.77
Fold 5: Accuracy =  0.91
Fold 6: Accuracy =  0.87
Fold 7: Accuracy =  0.92
Fold 8: Accuracy =  0.92
Fold 9: Accuracy =  0.92
Fold 10: Accuracy =  0.86
Average Accuracy: 0.88


## Naive Bayes

In [42]:
from sklearn.naive_bayes import GaussianNB

nbClassifier = GaussianNB()
nbClassifier.fit(X_train, y_train)

GaussianNB()

In [43]:
y_pred = nbClassifier.predict(X_test)

In [44]:
dTreeReportNB = pd.DataFrame(classification_report(y_test, y_pred, output_dict = True))
print(dTreeReportNB.T.style.to_latex())

\begin{tabular}{lrrrr}
 & precision & recall & f1-score & support \\
0 & 0.871308 & 0.860417 & 0.865828 & 480.000000 \\
1 & 0.636364 & 0.224359 & 0.331754 & 156.000000 \\
2 & 0.600000 & 0.310345 & 0.409091 & 29.000000 \\
3 & 0.182432 & 1.000000 & 0.308571 & 27.000000 \\
accuracy & 0.699422 & 0.699422 & 0.699422 & 0.699422 \\
macro avg & 0.572526 & 0.598780 & 0.478811 & 692.000000 \\
weighted avg & 0.780096 & 0.699422 & 0.704546 & 692.000000 \\
\end{tabular}



### using k-fold cross validation

In [45]:
from sklearn.model_selection import cross_val_score
k = 10
cross_val_results = cross_val_score(nbClassifier, X, y, cv = k)
for fold, accuracy in enumerate(cross_val_results, 1):
  print(f"Fold {fold}: Accuracy = {accuracy: .2f}")
print(f"Average Accuracy: {cross_val_results.mean():.2f}")

Fold 1: Accuracy =  0.73
Fold 2: Accuracy =  0.71
Fold 3: Accuracy =  0.74
Fold 4: Accuracy =  0.66
Fold 5: Accuracy =  0.69
Fold 6: Accuracy =  0.65
Fold 7: Accuracy =  0.67
Fold 8: Accuracy =  0.65
Fold 9: Accuracy =  0.65
Fold 10: Accuracy =  0.64
Average Accuracy: 0.68


## ANN

In [46]:
y.nunique()

4

In [47]:
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

# Transform data by standardization
sScaler = StandardScaler()
sX_train = sScaler.fit_transform(X_train)
sX_test = sScaler.fit_transform(X_test)



# building neural networks
model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation = 'relu', input_shape = (len(X.columns), )),
        tf.keras.layers.Dense(10, activation = 'relu'),
        tf.keras.layers.Dense(y.nunique(), activation = 'softmax'),
    ])

# compile the model
model.compile(optimizer = "adam", loss ="sparse_categorical_crossentropy", metrics=['accuracy'])

# Train the model
model.fit(sX_train, labeled_y_train, epochs = 100, batch_size = 16, verbose = 1)

Epoch 1/100
65/65 [==============================] - 1s 3ms/step - loss: 1.3127 - accuracy: 0.3832
Epoch 2/100
65/65 [==============================] - 0s 2ms/step - loss: 0.9823 - accuracy: 0.7027
Epoch 3/100
65/65 [==============================] - 0s 2ms/step - loss: 0.7859 - accuracy: 0.7056
Epoch 4/100
65/65 [==============================] - 0s 2ms/step - loss: 0.6683 - accuracy: 0.7066
Epoch 5/100
65/65 [==============================] - 0s 2ms/step - loss: 0.5941 - accuracy: 0.7239
Epoch 6/100
65/65 [==============================] - 0s 2ms/step - loss: 0.5424 - accuracy: 0.7568
Epoch 7/100
65/65 [==============================] - 0s 2ms/step - loss: 0.4988 - accuracy: 0.7838
Epoch 8/100
65/65 [==============================] - 0s 2ms/step - loss: 0.4628 - accuracy: 0.8127
Epoch 9/100
65/65 [==============================] - 0s 2ms/step - loss: 0.4314 - accuracy: 0.8224
Epoch 10/100
65/65 [==============================] - 0s 2ms/step - loss: 0.4055 - accuracy: 0.8388
Epoch 11/

In [48]:
y_pred = model.predict(sX_test)

22/22 [==============================] - 0s 2ms/step


In [49]:
y_pred_num = tf.argmax(y_pred, axis = 1).numpy()

In [50]:
named_y_pred = LabelEncoder().fit(y_train).inverse_transform(y_pred_num)

## Evaluation

In [51]:
dTreeReportANN = pd.DataFrame(classification_report(y_test, named_y_pred, output_dict = True))
print(dTreeReportANN.T.style.to_latex())

\begin{tabular}{lrrrr}
 & precision & recall & f1-score & support \\
0 & 0.985356 & 0.981250 & 0.983299 & 480.000000 \\
1 & 0.928105 & 0.910256 & 0.919094 & 156.000000 \\
2 & 0.750000 & 0.931034 & 0.830769 & 29.000000 \\
3 & 0.920000 & 0.851852 & 0.884615 & 27.000000 \\
accuracy & 0.958092 & 0.958092 & 0.958092 & 0.958092 \\
macro avg & 0.895865 & 0.918598 & 0.904444 & 692.000000 \\
weighted avg & 0.960036 & 0.958092 & 0.958582 & 692.000000 \\
\end{tabular}



In [128]:
dTreeReportIG.T['f1-score']

0               0.995825
1               0.948052
2               0.825397
3               0.800000
accuracy        0.969653
macro avg       0.892318
weighted avg    0.970272
Name: f1-score, dtype: float64

In [137]:

precisionTable = pd.DataFrame()
precisionTable["DTreeGRPrecision"] = dTreeReportIG.T['f1-score']
precisionTable["DTreeGiniPrecision"] = dTreeReportGini.T['f1-score']
precisionTable["NBPrecision"] = dTreeReportNB.T['f1-score']
precisionTable["ANNPrecision"] = dTreeReportANN.T['f1-score']


In [138]:
precisionTable.to_csv('precisionTable.csv')

In [139]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [140]:
# Evaluation Variables

dTreeReportIG.T.to_csv("dTreeReportIG.csv")
dTreeReportGini.T.to_csv("dTreeReportGini.csv")
dTreeReportNB.T.to_csv("dTreeReportNB.csv")
dTreeReportANN.T.to_csv("dTreeReportANN.csv")



In [141]:
precisionTable.columns

Index(['DTreeGRPrecision', 'DTreeGiniPrecision', 'NBPrecision',
       'ANNPrecision'],
      dtype='object')

In [142]:
precisionTable

,DTreeGRPrecision,DTreeGiniPrecision,NBPrecision,ANNPrecision
0,0.995825,0.996872,0.865828,0.983299
1,0.948052,0.947020,0.331754,0.919094
2,0.825397,0.787879,0.409091,0.830769
3,0.800000,0.842105,0.308571,0.884615
accuracy,0.969653,0.969653,0.699422,0.958092
macro avg,0.892318,0.893469,0.478811,0.904444
weighted avg,0.970272,0.970837,0.704546,0.958582


In [144]:
precisionTable.loc[[True]*4+[False]*3, 'DTreeGRPrecision']

0    0.995825
1    0.948052
2    0.825397
3    0.800000
Name: DTreeGRPrecision, dtype: float64

In [169]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sample accuracy, weighted average, and macro average values (replace with your actual data)
classifiers = ['Decision Tree w/ GR', 'Decision Tree w/ GI', 'Naive Bayes', 'ANN']
accuracy_values = [0.969653, 0.969653, 0.699422, 0.969653]
weighted_average = [0.970272, 0.970837,0.704546, 0.970272 ]  # Replace with actual weighted average
macro_average = [0.892318,0.893469,0.478811, 0.892318 ]  # Replace with actual macro average

# Create subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=['Accuracy', 'Weighted Avg', 'Macro Avg'])

# Create Scatter trace for Accuracy
trace_accuracy = go.Bar(
    x=classifiers,
    y=accuracy_values,
    # mode='markers+lines',
    marker=dict(color='darkgray'),
    # line=dict(color='black', width=2),
    text=['Accuracy: {:.2%}'.format(acc) for acc in accuracy_values],

    hoverinfo='text'
)

# Create Bar trace for Weighted Average
trace_weighted_average = go.Bar(
    x=classifiers,
    y=weighted_average,
    marker=dict(color='darkgray'),
    text=['Weighted Avg: {:.2%}'.format(w) for w in weighted_average],
    hoverinfo='text'
)

# Create Bar trace for Macro Average
trace_macro_average = go.Bar(
    x=classifiers,
    y=macro_average,
    marker=dict(color='darkgray'),
    text=['Macro Avg: {:.2%}'.format(m) for m in macro_average],
    hoverinfo='text'
)

# Add traces to subplots
fig.add_trace(trace_accuracy, row=1, col=1)
fig.add_trace(trace_weighted_average, row=1, col=2)
fig.add_trace(trace_macro_average, row=1, col=3)

# Update layout
fig.update_layout(
    title='Classifier Performance Comparison',
    showlegend=False,
    height=400,
    width=800,
    template = 'simple_white'
)

# Show the figure
fig.show()


In [146]:
# Precision values from your tables (replace these with your actual values)
precision_decision_tree_GainRatio = precisionTable.loc[[True]*4+[False]*3, 'DTreeGRPrecision']
precision_decision_tree_GiniIndex = precisionTable.loc[[True]*4+[False]*3, 'DTreeGiniPrecision']
precision_naive_bayes = precisionTable.loc[[True]*4+[False]*3, 'NBPrecision']
precision_ann = precisionTable.loc[[True]*4+[False]*3, 'ANNPrecision']

# Classes for the x-axis
classes = ['unacc', 'acc', 'good', 'vgood']

# Create traces for each classifier
trace_dt1 = go.Bar(x=classes, y=precision_decision_tree_GainRatio, name='Decision Tree using Gain Ratio',
                   marker = dict(
    color  = 'white', pattern_shape = 'x', line = dict(color = 'black', width = 2)
))
trace_dt2 = go.Bar(x=classes, y=precision_decision_tree_GiniIndex, name='Decision Tree using Gini Index',
                   marker = dict(
                       color  = 'white', pattern_shape = '/', line = dict(color = 'black', width = 2)
                   ))
trace_nb = go.Bar(x=classes, y=precision_naive_bayes, name='Naive Bayes', marker = dict(
    color  = 'white', pattern_shape = '|', line = dict(color = 'black', width = 2)
))
trace_ann = go.Bar(x=classes, y=precision_ann, name='ANN',marker = dict(
    color  = 'white', pattern_shape = '+', line = dict(color = 'black', width = 2)
))

# Layout settings
layout = go.Layout(title='F1-Score Comparison Across Classifiers',
                   xaxis=dict(title='Car Evaluation Classes'),
                   yaxis=dict(title='F1-Score'),
                   template = 'simple_white')

# Combine traces into a data list
data = [trace_dt1,trace_dt2, trace_nb, trace_ann]

# Create and show the figure
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# Precision values from your tables (replace these with your actual values)
precision_decision_tree_GainRatio = precisionTable.loc[[True]*4+[False]*3, 'DTreeGRPrecision']
precision_decision_tree_GiniIndex = precisionTable.loc[[True]*4+[False]*3, 'DTreeGiniPrecision']
precision_naive_bayes = precisionTable.loc[[True]*4+[False]*3, 'NBPrecision']
precision_ann = precisionTable.loc[[True]*4+[False]*3, 'ANNPrecision']

# Classes for the x-axis
classes = ['unacc', 'acc', 'good', 'vgood']

# Create traces for each classifier
trace_dt1 = go.Bar(x=classes, y=precision_decision_tree_GainRatio, name='Decision Tree using Gain Ratio',
                   marker = dict(
    color  = 'white', pattern_shape = 'x', line = dict(color = 'black', width = 2)
))
trace_dt2 = go.Bar(x=classes, y=precision_decision_tree_GiniIndex, name='Decision Tree using Gini Index',
                   marker = dict(
                       color  = 'white', pattern_shape = '/', line = dict(color = 'black', width = 2)
                   ))
trace_nb = go.Bar(x=classes, y=precision_naive_bayes, name='Naive Bayes', marker = dict(
    color  = 'white', pattern_shape = '|', line = dict(color = 'black', width = 2)
))
trace_ann = go.Bar(x=classes, y=precision_ann, name='ANN',marker = dict(
    color  = 'white', pattern_shape = '+', line = dict(color = 'black', width = 2)
))

# Layout settings
layout = go.Layout(title='Precision Comparison Across Classifiers',
                   xaxis=dict(title='Car Evaluation Classes'),
                   yaxis=dict(title='Precision'),
                   template = 'simple_white')

# Combine traces into a data list
data = [trace_dt1,trace_dt2, trace_nb, trace_ann]

# Create and show the figure
fig = go.Figure(data=data, layout=layout)
fig.show()

In [136]:
%%R
install.packages(readr)

  object 'readr' not found




Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  object 'readr' not found


In [65]:
%%R
library(tidyverse)
library(ggplot2)

dTreeReportIG <- read_csv("dTreeReportIG.csv")
dTreeReportGini <- read_csv("dTreeReportGini.csv")
dTreeReportNB <- read_csv("dTreeReportNB.csv")
dTreeReportANN <- read_csv("dTreeReportANN.csv")
precisionTable <- read_csv('precisionTable.csv')

New names:
• `` -> `...1`
Rows: 7 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): ...1
dbl (4): precision, recall, f1-score, support

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 7 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): ...1
dbl (4): precision, recall, f1-score, support

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 7 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): ...1
dbl (4): precision, recall, f1-score, support

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the 

In [56]:
dTreeReportIG.T

,precision,recall,f1-score,support
0,0.997908,0.993750,0.995825,480.000000
1,0.960526,0.935897,0.948052,156.000000
2,0.764706,0.896552,0.825397,29.000000
3,0.785714,0.814815,0.800000,27.000000
accuracy,0.969653,0.969653,0.969653,0.969653
macro avg,0.877214,0.910253,0.892318,692.000000
weighted avg,0.971429,0.969653,0.970272,692.000000


In [68]:
%%R
precisionTable

# A tibble: 7 × 5
  ...1         DTreeGRPrecision DTreeGiniPrecision NBPrecision ANNPrecision
  <chr>                   <dbl>              <dbl>       <dbl>        <dbl>
1 0                       0.998              0.998       0.871        0.985
2 1                       0.961              0.979       0.636        0.928
3 2                       0.765              0.703       0.6          0.75 
4 3                       0.786              0.8         0.182        0.92 
5 accuracy                0.970              0.970       0.699        0.958
6 macro avg               0.877              0.870       0.573        0.896
7 weighted avg            0.971              0.974       0.780        0.960


In [70]:
%%R
combinedPrecision <- ggplot(precisionTable) +
  geom_col(aes(DTreeGRPrecision))
print(combinedPrecision)

Error in `geom_col()`:
! Problem while setting up geom.
ℹ Error occurred in the 1st layer.
Caused by error in `compute_geom_1()`:
! `geom_col()` requires the following missing aesthetics: y
Run `rlang::last_trace()` to see where the error occurred.

Error in geom_col(aes(DTreeGRPrecision)) : 
ℹ Error occurred in the 1st layer.
Caused by error in `compute_geom_1()`:
! `geom_col()` requires the following missing aesthetics: y


In [58]:
%%R
combined_df <- rbind(
    transform(dTreeReportIG, dataset = "dTreeReportIG" ),
    transform(dTreeReportGini, dataset = "dTreeReportGini" ),
    transform(dTreeReportNB, dataset = "dTreeReportNB" ),
    transform(dTreeReportANN, dataset = "dTreeReportANN" )
)

combined_df

           ...1 precision    recall  f1.score     support         dataset
1             0 0.9979079 0.9937500 0.9958246 480.0000000   dTreeReportIG
2             1 0.9605263 0.9358974 0.9480519 156.0000000   dTreeReportIG
3             2 0.7647059 0.8965517 0.8253968  29.0000000   dTreeReportIG
4             3 0.7857143 0.8148148 0.8000000  27.0000000   dTreeReportIG
5      accuracy 0.9696532 0.9696532 0.9696532   0.9696532   dTreeReportIG
6     macro avg 0.8772136 0.9102535 0.8923184 692.0000000   dTreeReportIG
7  weighted avg 0.9714287 0.9696532 0.9702723 692.0000000   dTreeReportIG
8             0 0.9979123 0.9958333 0.9968717 480.0000000 dTreeReportGini
9             1 0.9794521 0.9166667 0.9470199 156.0000000 dTreeReportGini
10            2 0.7027027 0.8965517 0.7878788  29.0000000 dTreeReportGini
11            3 0.8000000 0.8888889 0.8421053  27.0000000 dTreeReportGini
12     accuracy 0.9696532 0.9696532 0.9696532   0.9696532 dTreeReportGini
13    macro avg 0.8700168 0.9244852 0.